# Calculating Probability time series
Author: Shalini Rome
Date: 6 Sept 2022

What I want to do: 

Files saved by location, merge all files of predictors based on year month and then you will have big data frame. Precip accumulation. Once have predictors with date for a location. Use your already saved RF (which random forest you use – can use any but don’t split data – want as much data as possible. Use model to predict probability at chosen location. Then add that column to the predictors/time dataframe. That is what I need to plot time series of that one. 

In [12]:
# Importing 
import pandas as pd

# Reading into pandas each predictors. I saved predictor files that had all locations and all years from 2000 to 2022. 
deep_drainage_df = pd.read_csv('All_Location_Deep_Drainage_test2022_07_29_21_07.csv') 
evap_df =  pd.read_csv('All_Location_E_Actual_test2022_07_29_21_07.csv') 
pet_df =  pd.read_csv('All_Location_PET_Actual_test2022_07_29_21_12.csv') 
qtot_df =  pd.read_csv('All_Location_Qtot_test2022_07_29_21_06.csv') 
rain_df =  pd.read_csv('All_Location_Rainfall_test2022_07_29_21_32.csv') 
soil_m_df =  pd.read_csv('All_Location_Soil_M_root_zone_test2022_07_29_21_06.csv') 

# I now need ENSO, IOD and SAMI and precip accum 3
ENSO = pd.read_csv('ENSO_1948_2022.csv', index = False) #This is from Sanaa on Slack. I converted to csv
IOD = pd.read_csv('IOD_DMI_1870_to_2022_Jan.csv') 
SAMI = pd.read_csv('SAMI_1957_to_2022_Mar.csv')
rain_accum_3 = pd.read_csv('rain_acc3M.csv', index = False)

TypeError: read_csv() got an unexpected keyword argument 'index'

For MoF's I need to take the excels and take the rain_df for example and collect ENSO, IOD and SAMI indexes from 2000 to 2022 as a time series. 

Used inspiration from this https://datacarpentry.org/python-ecology-lesson/05-merging-data/ How to identify columns that are the same

What I want to do is add only Qtot that is the row that is not in other dataframe. 

In [20]:
deep_drainage_df.info #.info

<bound method DataFrame.info of        Unnamed: 0   Location Year_Month  Deep_Drainage
0               0      Cobar    2000-01       2.416992
1               1      Cobar    2000-02       2.246094
2               2      Cobar    2000-03       2.930664
3               3      Cobar    2000-04       3.356445
4               4      Cobar    2000-05       4.364258
...           ...        ...        ...            ...
21971         263  Inglewood    2021-12       0.622070
21972         264  Inglewood    2022-01       0.658203
21973         265  Inglewood    2022-02       0.592773
21974         266  Inglewood    2022-03       0.661133
21975         267  Inglewood    2022-04       0.647461

[21976 rows x 4 columns]>

In [25]:
rain_df.info #.info

<bound method DataFrame.info of        Unnamed: 0   Location Year_Month    Rainfall
0               0      Cobar    2000-01    5.101562
1               1      Cobar    2000-02  124.703125
2               2      Cobar    2000-03   64.601562
3               3      Cobar    2000-04   76.203125
4               4      Cobar    2000-05   90.500000
...           ...        ...        ...         ...
21889         262  Inglewood    2021-11  197.000000
21890         263  Inglewood    2021-12  141.898438
21891         264  Inglewood    2022-01   69.703125
21892         265  Inglewood    2022-02  113.500000
21893         266  Inglewood    2022-03  131.000000

[21894 rows x 4 columns]>

In [6]:
qtot_df.columns # .info

<bound method DataFrame.info of        Unnamed: 0   Location Year_Month       Qtot
0               0      Cobar    2000-01   0.257812
1               1      Cobar    2000-02   1.507812
2               2      Cobar    2000-03   0.945312
3               3      Cobar    2000-04   0.781250
4               4      Cobar    2000-05   1.468750
...           ...        ...        ...        ...
21889         262  Inglewood    2021-11  17.640625
21890         263  Inglewood    2021-12  53.906250
21891         264  Inglewood    2022-01   0.117188
21892         265  Inglewood    2022-02   3.828125
21893         266  Inglewood    2022-03  25.328125

[21894 rows x 4 columns]>

In [47]:
merge_left = pd.merge(left = deep_drainage_df, right = evap_df, how = 'left')
#merge_left = merge_left.reset_index(drop=True)
merge_left = pd.merge(left = merge_left,right = pet_df, how = 'left') 
merge_left = pd.merge(left = merge_left, right = qtot_df, how = 'left')
merge_left = pd.merge(left = merge_left, right = rain_df, how = 'left')
merge_left = pd.merge(left = merge_left, right = soil_m_df, how = 'left')
merge_left = pd.merge(left = merge_left, right = rain_accum_3, how = 'left')
merge_left

,Unnamed: 0,Location,Year_Month,Deep_Drainage,E_Actual,PET_Actual,Qtot,Rainfall,Soil_M_root_zone,Year,Month,Longitude,Latitude,P_acc_3M
0,0,Cobar,2000-01,2.416992,85.445312,239.031250,0.257812,5.101562,0.273438,2000.0,1.0,145.8383,-31.4980,15.304688
1,1,Cobar,2000-02,2.246094,46.804688,184.265625,1.507812,124.703125,0.242188,2000.0,2.0,145.8383,-31.4980,134.906250
2,2,Cobar,2000-03,2.930664,78.593750,140.617188,0.945312,64.601562,0.396484,2000.0,3.0,145.8383,-31.4980,194.406250
3,3,Cobar,2000-04,3.356445,60.023438,97.171875,0.781250,76.203125,0.333984,2000.0,4.0,145.8383,-31.4980,265.507812
4,4,Cobar,2000-05,4.364258,43.335938,67.898438,1.468750,90.500000,0.416992,2000.0,5.0,145.8383,-31.4980,231.304688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22235,263,Inglewood,2021-12,0.622070,131.484375,214.718750,53.906250,141.898438,0.765625,2021.0,12.0,151.0833,-28.4167,391.398438
22236,264,Inglewood,2022-01,0.658203,122.679688,215.960938,0.117188,69.703125,0.514648,NaN,NaN,NaN,NaN,NaN
22237,265,Inglewood,2022-02,0.592773,77.781250,172.765625,3.828125,113.500000,0.439453,NaN,NaN,NaN,NaN,NaN
22238,266,Inglewood,2022-03,0.661133,96.398438,153.437500,25.328125,131.000000,0.530273,NaN,NaN,NaN,NaN,NaN


So the problem is rainfall and Qtot have less rows than the others. What is the difference? 

In [34]:
# Source: Comparing dataframes https://www.educative.io/answers/how-to-compare-two-dataframes-in-pandas 

# print(deep_drainage_df.compare(qtot_df,align_axis=0,keep_shape=False,keep_equal=False))
print(deep_drainage_df.compare(evap_df, keep_shape=True))
# Error: ValueError: Can only compare identically-labeled DataFrame objects

ValueError: Can only compare identically-labeled DataFrame objects

In [48]:
# Source: Using speedcoder5's answer - https://stackoverflow.com/questions/48647534/python-pandas-find-difference-between-two-data-frames 

df_1notin2 = deep_drainage_df[~(deep_drainage_df['Location'].isin(qtot_df['Location']) & deep_drainage_df['Year_Month'].isin(qtot_df['Year_Month']))]
df_1notin2

# It looks like April 2022 is the extra in the file. Which means that it is okay to have NaN for Rain and Qtot that do not consider April 2022. The one thing to note though is that I will need to remove the NaN.  

,Unnamed: 0,Location,Year_Month,Deep_Drainage
267,267,Cobar,2022-04,1.930664
535,267,Walgett,2022-04,0.256836
803,267,Coonamble,2022-04,0.093750
1071,267,Warren,2022-04,0.311523
1339,267,Nyngan,2022-04,0.785156
...,...,...,...,...
20903,267,Raymond Terrace,2022-04,21.745117
21171,267,Bogan,2022-04,0.785156
21439,267,Wellington,2022-04,0.000000
21707,267,Dungog,2022-04,5.822266


# Creating ENSO, IOD and SAM columns 

In [37]:
ENSO.columns = ['Year','January','February','March','April','May','June','July', 'August','September','October','November','December']
IOD.columns = ['Year','January','February','March','April','May','June','July', 'August','September','October','November','December']
SAMI.columns = ['Year','January','February','March','April','May','June','July', 'August','September','October','November','December']
# Note I have not removed -99. An example code is below:
#ENSO.replace(-99.0,'NaN',inplace = True)


Now the problem is the merge_left has only Year_Month and not Year sep from Month column. Thus we have to split it somehow

In [49]:
# Source: https://sparkbyexamples.com/pandas/pandas-extract-month-and-year-separately-from-datetime-column/
# Converting Year_Month to Year and Month for merge_left

# merge_left['Year'] = merge_left['Year_Month'].dt.year 
# merge_left['Month'] = merge_left['Year_Month'].dt.month
# merge_left
# Error: AttributeError: Can only use .dt accessor with datetimelike values
merge_left['Year'] = pd.DatetimeIndex(merge_left['Year_Month']).year
merge_left['Month'] = pd.DatetimeIndex(merge_left['Year_Month']).month # This converts it to a number
#merge_left
# Now converting Month to a name: https://stackoverflow.com/questions/37625334/python-pandas-convert-month-int-to-month-name
import calendar
merge_left['Month'] = merge_left['Month'].apply(lambda x: calendar.month_name[x]) # Month_abbr only gave half, so used other solution to see month_name gives full name. 
merge_left

,Unnamed: 0,Location,Year_Month,Deep_Drainage,E_Actual,PET_Actual,Qtot,Rainfall,Soil_M_root_zone,Year,Month,Longitude,Latitude,P_acc_3M
0,0,Cobar,2000-01,2.416992,85.445312,239.031250,0.257812,5.101562,0.273438,2000,January,145.8383,-31.4980,15.304688
1,1,Cobar,2000-02,2.246094,46.804688,184.265625,1.507812,124.703125,0.242188,2000,February,145.8383,-31.4980,134.906250
2,2,Cobar,2000-03,2.930664,78.593750,140.617188,0.945312,64.601562,0.396484,2000,March,145.8383,-31.4980,194.406250
3,3,Cobar,2000-04,3.356445,60.023438,97.171875,0.781250,76.203125,0.333984,2000,April,145.8383,-31.4980,265.507812
4,4,Cobar,2000-05,4.364258,43.335938,67.898438,1.468750,90.500000,0.416992,2000,May,145.8383,-31.4980,231.304688
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22235,263,Inglewood,2021-12,0.622070,131.484375,214.718750,53.906250,141.898438,0.765625,2021,December,151.0833,-28.4167,391.398438
22236,264,Inglewood,2022-01,0.658203,122.679688,215.960938,0.117188,69.703125,0.514648,2022,January,NaN,NaN,NaN
22237,265,Inglewood,2022-02,0.592773,77.781250,172.765625,3.828125,113.500000,0.439453,2022,February,NaN,NaN,NaN
22238,266,Inglewood,2022-03,0.661133,96.398438,153.437500,25.328125,131.000000,0.530273,2022,March,NaN,NaN,NaN


In [50]:
# Combining database and data tables according to Year,Month. 
clim_var_10 = []
clim_var_11 = []
clim_var_13 = []

row = 1

for row in merge_left.itertuples():
    year = row.Year
    mth = row.Month
    clim_var_10.append(float(ENSO.loc[(ENSO['Year']== year), mth]))
    clim_var_11.append(float(IOD.loc[(IOD['Year']== year), mth]))
    clim_var_13.append(float(SAMI.loc[(SAMI['Year']== year), mth]))

merge_left['ENSO'] = clim_var_10
merge_left['IOD'] = clim_var_11
merge_left['SAMI'] = clim_var_13

merge_left

,Unnamed: 0,Location,Year_Month,Deep_Drainage,E_Actual,PET_Actual,Qtot,Rainfall,Soil_M_root_zone,Year,Month,Longitude,Latitude,P_acc_3M,ENSO,IOD,SAMI
0,0,Cobar,2000-01,2.416992,85.445312,239.031250,0.257812,5.101562,0.273438,2000,January,145.8383,-31.4980,15.304688,-1.77,-0.060,3.59
1,1,Cobar,2000-02,2.246094,46.804688,184.265625,1.507812,124.703125,0.242188,2000,February,145.8383,-31.4980,134.906250,-1.55,0.071,2.32
2,2,Cobar,2000-03,2.930664,78.593750,140.617188,0.945312,64.601562,0.396484,2000,March,145.8383,-31.4980,194.406250,-0.98,0.240,0.74
3,3,Cobar,2000-04,3.356445,60.023438,97.171875,0.781250,76.203125,0.333984,2000,April,145.8383,-31.4980,265.507812,-0.87,0.217,0.97
4,4,Cobar,2000-05,4.364258,43.335938,67.898438,1.468750,90.500000,0.416992,2000,May,145.8383,-31.4980,231.304688,-0.86,0.192,1.90
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
22235,263,Inglewood,2021-12,0.622070,131.484375,214.718750,53.906250,141.898438,0.765625,2021,December,151.0833,-28.4167,391.398438,-1.06,-0.050,2.74
22236,264,Inglewood,2022-01,0.658203,122.679688,215.960938,0.117188,69.703125,0.514648,2022,January,NaN,NaN,NaN,-0.95,0.009,2.19
22237,265,Inglewood,2022-02,0.592773,77.781250,172.765625,3.828125,113.500000,0.439453,2022,February,NaN,NaN,NaN,-0.90,-9999.000,1.92
22238,266,Inglewood,2022-03,0.661133,96.398438,153.437500,25.328125,131.000000,0.530273,2022,March,NaN,NaN,NaN,-0.98,-9999.000,1.28


Now the output has both NaN values for April 2022 and -9999 or NaN values for 2022. 
Note: Precip accum datafile had more locations like city of wagga wagga

In [ ]:
# Droppping unecessary columns
merge_left.drop(['Unnamed: 0','Year','Month','Longitude','Latitude'],axis = 1,inplace=True)
merge_left

In [56]:
# Saving file 
merge_left.to_csv('Predictor_Time_Series_7Sept2022.csv')

In [58]:
merge_left

,Location,Year_Month,Deep_Drainage,E_Actual,PET_Actual,Qtot,Rainfall,Soil_M_root_zone,P_acc_3M,ENSO,IOD,SAMI
0,Cobar,2000-01,2.416992,85.445312,239.031250,0.257812,5.101562,0.273438,15.304688,-1.77,-0.060,3.59
1,Cobar,2000-02,2.246094,46.804688,184.265625,1.507812,124.703125,0.242188,134.906250,-1.55,0.071,2.32
2,Cobar,2000-03,2.930664,78.593750,140.617188,0.945312,64.601562,0.396484,194.406250,-0.98,0.240,0.74
3,Cobar,2000-04,3.356445,60.023438,97.171875,0.781250,76.203125,0.333984,265.507812,-0.87,0.217,0.97
4,Cobar,2000-05,4.364258,43.335938,67.898438,1.468750,90.500000,0.416992,231.304688,-0.86,0.192,1.90
...,...,...,...,...,...,...,...,...,...,...,...,...
22235,Inglewood,2021-12,0.622070,131.484375,214.718750,53.906250,141.898438,0.765625,391.398438,-1.06,-0.050,2.74
22236,Inglewood,2022-01,0.658203,122.679688,215.960938,0.117188,69.703125,0.514648,NaN,-0.95,0.009,2.19
22237,Inglewood,2022-02,0.592773,77.781250,172.765625,3.828125,113.500000,0.439453,NaN,-0.90,-9999.000,1.92
22238,Inglewood,2022-03,0.661133,96.398438,153.437500,25.328125,131.000000,0.530273,NaN,-0.98,-9999.000,1.28


# Random Forest classifier


Now that I have time series of predictors and locations. I make 

In [74]:
#Import dataset:
import pandas as pd
database = pd.read_csv('Database_All_AWRA_NDVI_MOf_and_3MPrecip_noWel_data_2022_08_12_16_48.csv')

# You can also remove duplicates. However, it is easier to do that in Excel manually. For the code currently we did not remove duplicates and kept them. 
# We did remove Wellington that was in NZ coordinates. In a previous version I removed Cobar that had NDVI and EVI -3000. But in this dataset I did not since we will not use NDVI and EVI in the end

# Dropping unecessary columns - IPO, NDVI and EVI
database.drop(['Unnamed: 0','Unnamed: 0.1','Year_Month','Latitude','Longitude', 'MYD13A3_061__1_km_monthly_EVI','MYD13A3_061__1_km_monthly_NDVI','MYD13A3_061__1_km_monthly_VI_Quality_VI_Usefulness_Description','IPO_TPI'],axis = 1,inplace=True)

# Dividing data into predictors (X) and no drought/drought (y)
y = database.iloc[:,4].values #All rows and only the 4th column (Drought no drought)
X = database 
X.drop(['Drought / No Drought'],axis = 1,inplace=True)
#Convert to array
X.to_numpy()

#Dividing data into training and testing: 
from sklearn.model_selection import train_test_split

# No splitting - use all data and set X_test_all to be the time_series
X_train_all = X
y_train = y
X_test_all = merge_left
X_test_all = X_test_all.dropna() # For the Value error below at classifier.predict - https://datascience.stackexchange.com/questions/11928/valueerror-input-contains-nan-infinity-or-a-value-too-large-for-dtypefloat32 
                # Realised I would rather drop rows with NAN than fill them with 0 as might come off as wrong.  https://datatofish.com/dropna/ 
 
# Then we create the real X_train and X_test by removing those columns. This allows us at the end to more easily see which ones were an issue later. 
X_train = X_train_all.drop(['Year','Month','Geographic Location','Location'],axis = 1)
X_test = X_test_all.drop(['Location','Year_Month'],axis = 1) 
# Since the column names should match - I will rearrange x_test.(https://erikrood.com/Python_References/change_order_dataframe_columns_final.html)
X_test = X_test[['Deep_Drainage', 'PET_Actual', 'E_Actual', 'Soil_M_root_zone', 'Qtot', 'Rainfall', 'ENSO', 'IOD', 'SAMI', 'P_acc_3M']]

# Training the model
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators=20, random_state=0)
classifier.fit(X_train, y_train)
y_pred_classifier = classifier.predict(X_test) #Error: ValueError: Input contains NaN, infinity or a value too large for dtype('float32').


# This section shows how the RF is predicting the drought compared to the binary values of 0 and 1. 
predictions = classifier.predict_proba(X_test)
# predictions # Col 1 is 0 and Col 2 is 1. We can
prob_drought = predictions[:,1]
# prob_drought
X_test_all['Probability_of_Drought'] = prob_drought
X_test_all

# Saving results
X_test_all.to_csv('RF_Probability_Time_Series_7Sept2022.csv')

# FutureWarning: The feature names should match those that were passed during fit. Starting version 1.2, an error will be raised. 

C:\Users\sy6sh\AppData\Local\Temp\ipykernel_3388\1315349049.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X_test_all['Probability_of_Drought'] = prob_drought


In [75]:
X_test_all

,Location,Year_Month,Deep_Drainage,E_Actual,PET_Actual,Qtot,Rainfall,Soil_M_root_zone,P_acc_3M,ENSO,IOD,SAMI,Probability_of_Drought
0,Cobar,2000-01,2.416992,85.445312,239.031250,0.257812,5.101562,0.273438,15.304688,-1.77,-0.060,3.59,0.30
1,Cobar,2000-02,2.246094,46.804688,184.265625,1.507812,124.703125,0.242188,134.906250,-1.55,0.071,2.32,0.10
2,Cobar,2000-03,2.930664,78.593750,140.617188,0.945312,64.601562,0.396484,194.406250,-0.98,0.240,0.74,0.00
3,Cobar,2000-04,3.356445,60.023438,97.171875,0.781250,76.203125,0.333984,265.507812,-0.87,0.217,0.97,0.00
4,Cobar,2000-05,4.364258,43.335938,67.898438,1.468750,90.500000,0.416992,231.304688,-0.86,0.192,1.90,0.10
...,...,...,...,...,...,...,...,...,...,...,...,...,...
22231,Inglewood,2021-08,0.600586,54.531250,97.648438,0.023438,12.398438,0.436523,149.500000,-0.53,0.014,0.73,0.20
22232,Inglewood,2021-09,0.576172,26.078125,128.664062,0.007812,8.296875,0.306641,98.695312,-0.55,0.048,1.68,0.20
22233,Inglewood,2021-10,0.580078,41.148438,173.460938,0.039062,52.500000,0.289062,73.195312,-0.94,0.068,0.80,0.05
22234,Inglewood,2021-11,0.551758,63.968750,168.187500,17.640625,197.000000,0.510742,257.796875,-0.94,0.192,2.69,0.00


In [71]:
X_test

,Deep_Drainage,PET_Actual,E_Actual,Soil_M_root_zone,Qtot,Rainfall,ENSO,IOD,SAMI,P_acc_3M
0,2.416992,239.031250,85.445312,0.273438,0.257812,5.101562,-1.77,-0.060,3.59,15.304688
1,2.246094,184.265625,46.804688,0.242188,1.507812,124.703125,-1.55,0.071,2.32,134.906250
2,2.930664,140.617188,78.593750,0.396484,0.945312,64.601562,-0.98,0.240,0.74,194.406250
3,3.356445,97.171875,60.023438,0.333984,0.781250,76.203125,-0.87,0.217,0.97,265.507812
4,4.364258,67.898438,43.335938,0.416992,1.468750,90.500000,-0.86,0.192,1.90,231.304688
...,...,...,...,...,...,...,...,...,...,...
22231,0.600586,97.648438,54.531250,0.436523,0.023438,12.398438,-0.53,0.014,0.73,149.500000
22232,0.576172,128.664062,26.078125,0.306641,0.007812,8.296875,-0.55,0.048,1.68,98.695312
22233,0.580078,173.460938,41.148438,0.289062,0.039062,52.500000,-0.94,0.068,0.80,73.195312
22234,0.551758,168.187500,63.968750,0.510742,17.640625,197.000000,-0.94,0.192,2.69,257.796875


In [62]:
X_test.columns

Index(['Deep_Drainage', 'E_Actual', 'PET_Actual', 'Qtot', 'Rainfall',
       'Soil_M_root_zone', 'P_acc_3M', 'ENSO', 'IOD', 'SAMI'],
      dtype='object')

In [61]:
X_train.columns

Index(['Deep_Drainage', 'PET_Actual', 'E_Actual', 'Soil_M_root_zone', 'Qtot',
       'Rainfall', 'ENSO', 'IOD', 'SAMI', 'P_acc_3M'],
      dtype='object')

THUS NOW WE HAVE XTEST - 21648 rows × 13 columns


# Creating animation of locations   
https://realpython.com/image-processing-with-the-python-pillow-library/

In [88]:
# Using Pillow
from PIL import Image, ImageDraw, ImageChops, ImageFont

filename = 'Prob_fig_Cobar_9Sept2022.png'
with Image.open(filename) as img: #open() reads the image from the file and .load() read the image into memory.
    img.load()

type(img)
# PIL.PngImagePlugin.PngImageFile

isinstance(img, Image.Image) #It is a subclass of Image Class. 
# True

#img.show
img.format #PNG
img.size # (432, 576)
img.mode #RGBA


'RGBA'

Below is the working code

In [111]:
# Using Pillow
from PIL import Image
import os
import glob

##Setting path of files and giving name
files = os.path.join("Prob_fig_*.png")
##List of merged files returned
images = glob.glob(files)

#for image in images:
    
 #   with Image.open(image) as img: #open() reads the image from the file and .load() read the image into memory.
  #      img.load()

# Solution I want to get all the images and append them to each other : https://stackoverflow.com/questions/67612802/using-pil-to-generate-gif-from-many-png-files 
fp_out = "Monthly_Drought_Probability_9Sept2022.gif"
fp_out_fast = 'Monthly_Drought_Probability_9Sept2022_fast.gif'
fp_out_slow = 'Monthly_Drought_Probability_9Sept2022_slow.gif'

# https://pillow.readthedocs.io/en/stable/handbook/image-file-formats.html#gif
img, *imgs = [Image.open(f) for f in sorted(images)]
img.save(fp=fp_out, format='GIF', append_images=imgs,
         save_all=True, duration=1000, loop=0)

img.save(fp=fp_out_fast, format='GIF', append_images=imgs,
         save_all=True, duration=200, loop=0)

img.save(fp=fp_out_slow, format='GIF', append_images=imgs,
         save_all=True, duration=2000, loop=0)

        
# Another few links that were useful to explain the code:
#  https://www.pythoninformer.com/python-libraries/pillow/creating-animated-gif/



The images need to have all the same legends from 0 to 1. 